<a href="https://colab.research.google.com/github/AdamKimhub/Msproject1/blob/colab/forfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    dataset_path = '/content/drive/My Drive/dataset'

    # Install required packages
    !pip install pymatgen

else:
    dataset_path = 'dataset'

In [9]:
from pathlib import Path
import ast
import numpy as np
import pandas as pd
from pymatgen.core import Structure, PeriodicSite, DummySpecie
from pymatgen.analysis.local_env import MinimumDistanceNN


## When  dealing with high dataset only

In [ ]:
high_dataset = ["high_BN", "high_GaSe", "high_InSe", "high_MoS2", "high_P", "high_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{high_data}.csv") for high_data in high_dataset]

high_df  = pd.concat(to_merge, ignore_index=True)

high_copy = high_df.copy()

In [ ]:
high_copy = high_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
high_copy = high_copy.corr()
high_copy

## When dealing with low dataset

In [ ]:
low_dataset = ["low_MoS2", "low_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{low_data}.csv") for low_data in low_dataset]

low_df  = pd.concat(to_merge, ignore_index=True)

low_copy = low_df.copy()

In [ ]:
low_copy = low_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
low_copy = low_copy.corr()
low_copy

## When dealing with both low and high dataset

In [3]:
# Read file
sample_df = pd.read_csv(f"{dataset_path}/combined/combined.csv")

In [4]:
sample_copy = sample_df.copy()
sample_copy = sample_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
sample_copy = sample_copy.corr()
sample_copy

,Unnamed: 0,energy,formation_energy,energy_per_atom,E_1,band_gap_value,vacancy_sites,substitution_sites,defect_sites
Unnamed: 0,1.000000,-0.543755,-0.459145,-0.348285,0.060681,0.075284,-0.455989,-0.486571,-0.513516
energy,-0.543755,1.000000,0.291578,0.876757,0.498878,-0.065589,0.500522,0.481216,0.534796
formation_energy,-0.459145,0.291578,1.000000,-0.058278,0.188604,-0.450856,0.756871,0.524373,0.697688
energy_per_atom,-0.348285,0.876757,-0.058278,1.000000,0.388663,-0.027159,0.351724,0.344729,0.379397
E_1,0.060681,0.498878,0.188604,0.388663,1.000000,-0.427162,0.226652,0.048636,0.149743
band_gap_value,0.075284,-0.065589,-0.450856,-0.027159,-0.427162,1.000000,-0.470108,-0.017354,-0.264983
vacancy_sites,-0.455989,0.500522,0.756871,0.351724,0.226652,-0.470108,1.000000,0.684788,0.917422
substitution_sites,-0.486571,0.481216,0.524373,0.344729,0.048636,-0.017354,0.684788,1.000000,0.918218
defect_sites,-0.513516,0.534796,0.697688,0.379397,0.149743,-0.264983,0.917422,0.918218,1.000000


## Data to graphs

In [10]:
samplex_df = pd.read_csv(f"{dataset_path}/combined/high_GaSe.csv")

In [11]:
import to_graph

def graphy(row):
    defective_file_path = Path(f"{dataset_path}/{row['dataset_material']}/cifs/{row['_id']}.cif")
    defective_structure = Structure.from_file(defective_file_path)

    ref_file_path = Path(f"{dataset_path}/{row['dataset_material']}/{row['base']}.cif")
    ref_unit_cell = Structure.from_file(ref_file_path)
    the_cell = ast.literal_eval(row['cell'])
    reference_structure = ref_unit_cell.make_supercell(the_cell)

    defects_structure = to_graph.get_defects_structure(defective_structure, reference_structure)

    # the_nodes, the_edges, the_edge_features = to_graph.get_nodes_edges(defects_structure)

    # global_attributes = ["energy","fermi_level","total_mag","formation_energy",
                         # "energy_per_atom","E_1","vacancy_sites",
                         # "substitution_sites","defect_sites"]
    
    # global_features = [row[i] for i in global_attributes]
    
    # target_attribute = "band_gap_value"
    # target_features = [row[target_attribute]]

    # return the_nodes, the_edges, the_edge_features, global_features, target_features
    return defects_structure

samplex = samplex_df.apply(lambda row: graphy(row), axis = 1)

/home/adam/anaconda3/envs/MsProject/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [14]:
print(samplex[0])

Full Formula (X1 In1 S1)
Reduced Formula: XInS
abc   :  22.914400  22.914400  20.000000
angles:  90.000000  90.000000 120.000012
pbc   :       True       True       True
Sites (3)
  #  SP           a         b         c    an_change    new_an    original_an    substitution_defect    vacancy_defect
---  ----  --------  --------  --------  -----------  --------  -------------  ---------------------  ----------------
  0  In    0.111111  0.388889  0.383273           18        49             31                      1                 0
  1  S     0.388889  0.944444  0.56548           -18        16             34                      1                 0
  2  X0+   0.056     0.111     0.325             -34         0             34                      0                 1
